# Kinetic temperature

The kinetic temperature is obtained with the 12CO amplitude :
$$T_{kin}=T_{x}=\frac{5.532}{\ln\left(1+\left(\frac{T_A}{5.532}+0.151\right)^{-1}\right)}$$

In [ ]:
from src.hdu.maps.map import Map
from src.hdu.cubes.cube_co import CubeCO
from src.hdu.tesseract import Tesseract
from src.hdu.maps.grouped_maps import GroupedMaps
from src.hdu.maps.convenient_funcs import get_kinetic_temperature
from src.coordinates.ds9_coords import DS9Coords

In [ ]:
def compute_kinetic_temperature(prefix: str):
    GroupedMaps([(
        "kinetic_temperature", [
            get_kinetic_temperature(amp) for amp in Tesseract.load(
                f"data/Loop4_co/{prefix}/12co/object_filtered.fits"
            ).to_grouped_maps().amplitude
        ]
    )]).save(f"data/Loop4_co/{prefix}/12co/kinetic_temperature.fits")

In [ ]:
compute_kinetic_temperature("N1")

In [ ]:
compute_kinetic_temperature("N2")

In [ ]:
compute_kinetic_temperature("N4")

In [ ]:
compute_kinetic_temperature("p")

# Column density

The column density is obtained using the following equation (Interstellar And Intergalactic Medium, Barbara Ryden and Richard W. Pogge):
\begin{align*}
    N_0\left(^{13}\text{CO}\right)=\int_{-\infty}^\infty T_A\left(^{13}\text{CO}\right)\cdot0.8\cdot\frac{g_0}{g_1A_{10}}\cdot\frac{\pi k\nu^2}{hc^3}\left[\left(\frac1{\exp\left(\frac{h\nu}{kT_x}\right)-1}-\frac1{\exp\left(\frac{h\nu}{kT_{rad}}\right)-1}\right)\left(1-\exp\left(-\frac{h\nu}{kT_x}\right)\right)\right]^{-1}
\end{align*}
knowing that
\begin{align*}
    \int_{-\infty}^\infty T_A\left(^{13}\text{CO}\right)dv&=2T_A\sigma\sqrt{\frac\pi2}\text{erf}\left(\frac{\infty}{\sqrt2\sigma}\right)\\
    &=2T_A\sigma\sqrt{\frac\pi2}\\
\end{align*}
as
$$\lim_{x\rightarrow\infty}\text{erf}(x)=1$$
we obtain
$$N_0\left(^{13}\text{CO}\right)=2T_A\sigma\sqrt{\frac\pi2}\cdot0.8\cdot\frac{g_0}{g_1A_{10}}\cdot\frac{\pi k\nu^2}{hc^3}\left[\left(\frac1{\exp\left(\frac{h\nu}{kT_x}\right)-1}-\frac1{\exp\left(\frac{h\nu}{kT_{rad}}\right)-1}\right)\left(1-\exp\left(-\frac{h\nu}{kT_x}\right)\right)\right]^{-1}$$

In [ ]:
import numpy as np
import graphinglib as gl
import importlib
import scipy

import src.hdu.maps.convenient_funcs
importlib.reload(src.hdu.maps.convenient_funcs)

In [ ]:
def compute_single_component_column_density(prefix: str):
    cube_12co = CubeCO.load(f"data/Loop4_co/{prefix}/12co/Loop4{prefix}_wcs.fits")
    cube_13co = CubeCO.load(f"data/Loop4_co/{prefix}/13co/Loop4{prefix}_13co.fits")
    maps_12co = Tesseract.load(f"data/Loop4_co/{prefix}/12co/object_filtered.fits").to_grouped_maps()
    maps_13co = Tesseract.load(f"data/Loop4_co/{prefix}/13co/tesseract.fits").to_grouped_maps()

    # The right gaussians first need to be selected
    # This solution is for single component 13co maps
    assert len(maps_13co.mean) == 1
    mean_12co = np.stack([m.get_reprojection_on(maps_13co.mean[0]).data for m in maps_12co.mean], axis=0)
    offset_12 = sum([int(line.split(" ")[5][:-1]) if line[12:33] == "was sliced at channel" else 0 
                     for line in maps_12co.mean[0].header["COMMENT"]])
    offset_13 = sum([int(line.split(" ")[5][:-1]) if line[13:34] == "was sliced at channel" else 0 
                     for line in maps_13co.mean[0].header["COMMENT"]])

    speed_convert_12 = np.vectorize(cube_12co.header.get_value)
    speed_convert_13 = np.vectorize(cube_13co.header.get_value)
    # Compute the diff between the centroid of every gaussian
    diff_array = np.abs(speed_convert_12(mean_12co + offset_12)
                      - speed_convert_13(maps_13co.mean[0].data + offset_13))
    nan_mask = np.isnan(diff_array)     # Apply a nan mask to allow proper argmin use
    diff_array[nan_mask] = 2**15-1      # Remove nans
    min_mask = np.argmin(diff_array, axis=0)
    filter_gaussians = lambda arr: np.take_along_axis(arr, min_mask[np.newaxis, ...], axis=0).squeeze()

    amp_12co_val = np.stack(
        [m.get_reprojection_on(maps_13co.mean[0]).data for m in maps_12co.amplitude], axis=0
    )
    amp_12co_unc = np.stack(
        [m.get_reprojection_on(maps_13co.mean[0]).uncertainties for m in maps_12co.amplitude], axis=0
    )

    amplitude_correction_factor_13co = 0.43
    src.hdu.maps.convenient_funcs.get_13co_column_density(
        stddev_13co=maps_13co.stddev[0]*np.abs(cube_13co.header["CDELT3"]/1000),
        antenna_temperature_13co=maps_13co.amplitude[0]/amplitude_correction_factor_13co,
        antenna_temperature_12co=Map(filter_gaussians(amp_12co_val), filter_gaussians(amp_12co_unc))
    ).save(f"data/Loop4_co/{prefix}/13co/{prefix}_column_density.fits")

In [ ]:
compute_single_component_column_density("N1")

In [ ]:
compute_single_component_column_density("N2")

In [ ]:
compute_single_component_column_density("N4")

In [ ]:
# Loop4p multiple components
import scipy.optimize

cube_12co = CubeCO.load("data/Loop4_co/p/12co/Loop4p_wcs.fits")
cube_13co = CubeCO.load("data/Loop4_co/p/13co/Loop4p_13co.fits")
maps_12co = Tesseract.load("data/Loop4_co/p/12co/object_filtered.fits").to_grouped_maps()
maps_13co = Tesseract.load("data/Loop4_co/p/13co/object_filtered.fits").to_grouped_maps()

mean_12co = np.stack([m.get_reprojection_on(maps_13co.mean[0]).data for m in maps_12co.mean], axis=0)
mean_13co = np.stack([m.data for m in maps_13co.mean], axis=0)
ampl_12co_val = np.stack([m.get_reprojection_on(maps_13co.amplitude[0]).data for m in maps_12co.amplitude], axis=0)
ampl_12co_unc = np.stack([m.get_reprojection_on(maps_13co.amplitude[0]).uncertainties for m in maps_12co.amplitude],
                         axis=0)

ordered_stddev_13co = np.full([*mean_13co.shape, 2], np.NAN)
ordered_amplitude_13co = np.full([*mean_13co.shape, 2], np.NAN)
ordered_amplitude_12co = np.full([*mean_13co.shape, 2], np.NAN)

speed_convert_12 = np.vectorize(cube_12co.header.get_value)
speed_convert_13 = np.vectorize(cube_13co.header.get_value)

def minimize(target: np.ndarray, ref: np.ndarray):
    """ 
    Minimizes the distance between two groups of points and gives the matching indices.
    """
    # Create a cost matrix where the element at position (i, j) represents the difference between list1[i] and list2[j]
    cost_matrix = np.abs(np.subtract.outer(target[~np.isnan(target)], ref[~np.isnan(ref)]))
    # Use linear_sum_assignment to find the optimal assignment
    row_indices, col_indices = scipy.optimize.linear_sum_assignment(cost_matrix)
    # Create a list of tuples representing the pairs
    pairs = list(zip(row_indices, col_indices))
    # Check if the pairs are close enough, otherwise the pair is considered invalid
    velocity_upper_limit = 100
    valid_pairs = []
    for pair in pairs:
        if np.abs(target[pair[0]] - ref[pair[1]]) < velocity_upper_limit:
            valid_pairs.append(pair)
    return valid_pairs

for y in range(mean_13co.shape[1]):
    for x in range(mean_13co.shape[2]):
        if not np.isnan(mean_13co[0,y,x]):
            matches = minimize(speed_convert_13(mean_13co[:,y,x]+400), speed_convert_12(mean_12co[:,y,x]+500))
            for match in matches:
                ordered_stddev_13co[match[0],y,x] = [
                    maps_13co.stddev[match[0]].data[y,x],
                    maps_13co.stddev[match[0]].uncertainties[y,x]
                ]
                ordered_amplitude_13co[match[0],y,x] = [
                    maps_13co.amplitude[match[0]].data[y,x],
                    maps_13co.amplitude[match[0]].uncertainties[y,x]
                ]
                ordered_amplitude_12co[match[0],y,x] = [
                    ampl_12co_val[match[1],y,x],
                    ampl_12co_unc[match[1],y,x]
                ]

amplitude_correction_factor_13co = 0.43
column_densities = []
for i in range(mean_13co.shape[0]):
    column_densities.append(
        src.hdu.maps.convenient_funcs.get_13co_column_density(
            stddev_13co=Map(
                data=ordered_stddev_13co[i,:,:,0],
                uncertainties=ordered_stddev_13co[i,:,:,1],
                header=maps_13co.mean[0].header,
            ) * np.abs(cube_13co.header["CDELT3"]/1000),
            antenna_temperature_13co=Map(
                data=ordered_amplitude_13co[i,:,:,0],
                uncertainties=ordered_amplitude_13co[i,:,:,1],
                header=maps_13co.mean[0].header,
            ) / amplitude_correction_factor_13co,
            antenna_temperature_12co=Map(
                data=ordered_amplitude_12co[i,:,:,0],
                uncertainties=ordered_amplitude_12co[i,:,:,1],
                header=maps_13co.mean[0].header,
            )
        )
    )

GroupedMaps([("column_density", column_densities)]).save("data/Loop4_co/p/13co/p_column_density.fits")

## H2 column density

In [ ]:
def calculate_h2_column_density(
        prefix: str,
):
    tess = Tesseract.load(f"data/Loop4_co/{prefix}/12co/object_filtered.fits")
    cube = CubeCO.load(f"data/Loop4_co/{prefix}/12co/Loop4{prefix}_wcs.fits")
    X_CO = 2.5e20
    gm = tess.to_grouped_maps()
    n_h2 = []
    for amplitude, stddev in zip(gm.amplitude, gm.stddev):
        n_h2.append(
            src.hdu.maps.convenient_funcs.integrate_gaussian(
                amplitude_map=amplitude,
                stddev_map=stddev * np.abs(cube.header["CDELT3"]) / 1000
            ) * X_CO
        )
    GroupedMaps([("H2_column_density", n_h2)]).save(f"data/Loop4_co/{prefix}/12co/{prefix}_H2_column_density.fits")
    Map(
        data=np.nansum([m.data for m in n_h2], axis=0),
        uncertainties=np.nansum([m.uncertainties for m in n_h2], axis=0),
        header=n_h2[0].header,
    ).num_to_nan().save(f"data/Loop4_co/{prefix}/12co/{prefix}_H2_column_density_total.fits")

def calculate_h2_column_density_with_13co(
        prefix: str,
):
    if prefix == "p":
        column_densities = GroupedMaps.load("data/Loop4_co/p/13co/p_column_density.fits").column_density
        (2.2e6 * Map(
            data=np.nansum([m.data for m in column_densities], axis=0),
            uncertainties=np.nansum([m.uncertainties for m in column_densities], axis=0),
            header=column_densities[0].header,
        ).num_to_nan()).save(f"data/Loop4_co/{prefix}/13co/{prefix}_H2_column_density_total_13co.fits")
        
    else:
        column_density = Map.load(f"data/Loop4_co/{prefix}/13co/{prefix}_column_density.fits")
        (2.2e6 * column_density).save(f"data/Loop4_co/{prefix}/13co/{prefix}_H2_column_density_total_13co.fits")

In [ ]:
calculate_h2_column_density("N1")
calculate_h2_column_density("N2")
calculate_h2_column_density("N4")
calculate_h2_column_density("p")

In [ ]:
calculate_h2_column_density_with_13co("N1")
calculate_h2_column_density_with_13co("N2")
calculate_h2_column_density_with_13co("N4")
calculate_h2_column_density_with_13co("p")

## Cloud mass

In [ ]:
def calculate_cloud_mass(
        prefix: str,
):
    """
    Gives the cloud's mass in kg.
    """
    alpha = 30 / 3600 * (2*np.pi)/360 
    if prefix in ["N1", "p"]:       # These two clouds were binned 2x2 whuch results in 
        alpha *= 2
    D = 370 * scipy.constants.parsec * 100
    mu = 2.4
    m_H = scipy.constants.proton_mass + scipy.constants.electron_mass

    n_h2 = Map.load(f"data/Loop4_co/{prefix}/12co/{prefix}_H2_column_density_total.fits")
    sum_n_h2 = np.array([
        np.nansum(n_h2.data),
        np.nansum(n_h2.uncertainties),
    ])
    M = (alpha * D)**2 * mu * m_H * sum_n_h2
    return M

In [ ]:
fm = lambda x: f"({x[0]:.5e} ± {x[1]:.5e})"
for cloud in ["N1", "N2", "N4", "p"]:
    m = calculate_cloud_mass(cloud)
    print(f"Cloud {cloud:2}: M(H2)={fm(m):27} kg, M(CO)={fm(3e-6 * m):27} kg")